# Gold Layer Aggregation - Databricks

This notebook creates business-ready aggregate tables for analytics and ML.

**Prerequisites:**
- Silver layer transformation completed
- Tables exist in `/FileStore/instacart/silver/`

**Output:**
- Business metrics in `/FileStore/instacart/gold/`

In [ ]:
# Configuration
SILVER_PATH = "/FileStore/instacart/silver"
GOLD_PATH = "/FileStore/instacart/gold"

print(f"Silver input path: {SILVER_PATH}")
print(f"Gold output path: {GOLD_PATH}")
print(f"Spark version: {spark.version}")

## Metric 1: Product Performance Metrics

Calculate order frequency, reorder rate, and avg cart position per product.

In [ ]:
from pyspark.sql.functions import count, sum as spark_sum, avg, countDistinct
from pyspark.sql.functions import round as spark_round, current_timestamp, col, desc

# Read Silver data
order_products_df = spark.read.format("delta").load(f"{SILVER_PATH}/order_products_prior_enriched")

# Calculate product metrics
product_metrics = order_products_df \
    .groupBy("product_id", "product_name", "aisle", "department") \
    .agg(
        count("order_id").alias("total_orders"),
        countDistinct("user_id").alias("unique_customers"),
        spark_sum("reordered").alias("reorder_count"),
        avg("reordered").alias("reorder_rate"),
        avg("add_to_cart_order").alias("avg_cart_position")
    ) \
    .withColumn("reorder_rate", spark_round(col("reorder_rate"), 4)) \
    .withColumn("avg_cart_position", spark_round(col("avg_cart_position"), 2)) \
    .orderBy(col("total_orders").desc()) \
    .withColumn("processing_timestamp", current_timestamp())

# Write to Gold
product_metrics.write \
    .format("delta") \
    .mode("overwrite") \
    .save(f"{GOLD_PATH}/product_metrics")

count = product_metrics.count()
print(f"✓ Created product_metrics with {count:,} products")

In [ ]:
# Preview top products
print("Top 20 Products by Order Count:")
display(product_metrics.select("product_name", "total_orders", "reorder_rate", "unique_customers").limit(20))

## Metric 2: Department Performance

Aggregate department-level metrics for business reporting.

In [ ]:
# Calculate department metrics
department_metrics = order_products_df \
    .groupBy("department_id", "department") \
    .agg(
        count("order_id").alias("total_orders"),
        countDistinct("product_id").alias("unique_products"),
        countDistinct("user_id").alias("unique_customers"),
        spark_sum("reordered").alias("reorder_count"),
        avg("reordered").alias("avg_reorder_rate")
    ) \
    .withColumn("avg_reorder_rate", spark_round(col("avg_reorder_rate"), 4)) \
    .orderBy(col("total_orders").desc()) \
    .withColumn("processing_timestamp", current_timestamp())

# Write to Gold
department_metrics.write \
    .format("delta") \
    .mode("overwrite") \
    .save(f"{GOLD_PATH}/department_metrics")

count = department_metrics.count()
print(f"✓ Created department_metrics with {count:,} departments")

In [ ]:
# Preview department performance
print("Department Performance:")
display(department_metrics.select("department", "total_orders", "unique_products", "avg_reorder_rate"))

## Metric 3: User Purchase Features

Create ML-ready features for customer segmentation.

In [ ]:
# Read user summary
user_summary_df = spark.read.format("delta").load(f"{SILVER_PATH}/user_order_summary")

# Calculate user purchase features
user_features = order_products_df \
    .groupBy("user_id") \
    .agg(
        countDistinct("order_id").alias("total_orders"),
        countDistinct("product_id").alias("unique_products_purchased"),
        count("product_id").alias("total_items_purchased"),
        avg("reordered").alias("reorder_propensity"),
        countDistinct("department").alias("departments_shopped")
    ) \
    .join(user_summary_df, "user_id", "left") \
    .withColumn("avg_basket_size", 
                spark_round(col("total_items_purchased") / col("total_orders"), 2)) \
    .withColumn("reorder_propensity", spark_round(col("reorder_propensity"), 4)) \
    .select(
        "user_id",
        "total_orders",
        "unique_products_purchased",
        "total_items_purchased",
        "avg_basket_size",
        "reorder_propensity",
        "departments_shopped",
        "avg_order_dow",
        "avg_order_hour",
        "avg_days_between_orders"
    ) \
    .withColumn("processing_timestamp", current_timestamp())

# Write to Gold
user_features.write \
    .format("delta") \
    .mode("overwrite") \
    .save(f"{GOLD_PATH}/user_purchase_features")

count = user_features.count()
print(f"✓ Created user_purchase_features with {count:,} users")

In [ ]:
# Preview user features
print("User Purchase Features - Sample:")
display(user_features.limit(20))

## Metric 4: Product Pairs Affinity (Basket Analysis)

Find products frequently bought together.

In [ ]:
# Widget for min support (Databricks-specific)
dbutils.widgets.text("min_support", "100", "Minimum Support")
min_support = int(dbutils.widgets.get("min_support"))

print(f"Minimum support threshold: {min_support}")

In [ ]:
# Self-join to find product pairs
product_pairs = order_products_df.alias("a") \
    .join(
        order_products_df.alias("b"),
        (col("a.order_id") == col("b.order_id")) & (col("a.product_id") < col("b.product_id")),
        "inner"
    ) \
    .select(
        col("a.product_id").alias("product_a_id"),
        col("a.product_name").alias("product_a_name"),
        col("b.product_id").alias("product_b_id"),
        col("b.product_name").alias("product_b_name")
    ) \
    .groupBy("product_a_id", "product_a_name", "product_b_id", "product_b_name") \
    .agg(count("*").alias("pair_count")) \
    .filter(col("pair_count") >= min_support) \
    .orderBy(col("pair_count").desc()) \
    .withColumn("processing_timestamp", current_timestamp())

# Write to Gold
product_pairs.write \
    .format("delta") \
    .mode("overwrite") \
    .save(f"{GOLD_PATH}/product_pairs_affinity")

count = product_pairs.count()
print(f"✓ Created product_pairs_affinity with {count:,} product pairs")

In [ ]:
# Preview top product pairs
print(f"Top 20 Product Pairs (Frequently Bought Together, min_support={min_support}):")
display(product_pairs.select("product_a_name", "product_b_name", "pair_count").limit(20))

## Verify Gold Tables

List all created Gold tables and their record counts.

In [ ]:
# List Gold tables
gold_tables = dbutils.fs.ls(GOLD_PATH)

print("=" * 80)
print("GOLD LAYER AGGREGATION COMPLETE")
print("=" * 80)
print("\nGold tables created:")
for table in gold_tables:
    print(f"  ✓ {table.name}")

# Display record counts
print("\nRecord counts:")
print(f"  - product_metrics: {spark.read.format('delta').load(f'{GOLD_PATH}/product_metrics').count():,}")
print(f"  - department_metrics: {spark.read.format('delta').load(f'{GOLD_PATH}/department_metrics').count():,}")
print(f"  - user_purchase_features: {spark.read.format('delta').load(f'{GOLD_PATH}/user_purchase_features').count():,}")
print(f"  - product_pairs_affinity: {spark.read.format('delta').load(f'{GOLD_PATH}/product_pairs_affinity').count():,}")

## Query Gold Tables with SQL

Use Databricks SQL to query the Gold tables.

In [ ]:
-- Top 10 products by total orders
SELECT 
  product_name,
  total_orders,
  reorder_rate,
  unique_customers
FROM delta.`/FileStore/instacart/gold/product_metrics`
ORDER BY total_orders DESC
LIMIT 10

In [ ]:
-- Department performance summary
SELECT 
  department,
  total_orders,
  unique_products,
  avg_reorder_rate
FROM delta.`/FileStore/instacart/gold/department_metrics`
ORDER BY total_orders DESC

## Summary

✅ **Gold layer aggregation complete!**

**Tables created:**
- `product_metrics` - Product performance KPIs
- `department_metrics` - Category-level analytics
- `user_purchase_features` - Customer segmentation features (ML-ready)
- `product_pairs_affinity` - Basket analysis (frequently bought together)

**Use cases:**
- **BI Dashboards**: Connect Tableau/Power BI to Gold tables
- **ML Models**: Use user_purchase_features for segmentation
- **Recommendations**: Use product_pairs_affinity for cross-sell
- **Business Reporting**: Query metrics with SQL

**Next steps:**
1. Run `instacart_analysis_databricks` for interactive exploration
2. Create Databricks SQL dashboards
3. Build ML models with MLflow